In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
item_df = pd.read_csv('item.csv')
alias_df = pd.read_csv('item_aliases.csv')
#page_df = pd.read_csv('page.csv')

CPU times: user 49.8 s, sys: 4.69 s, total: 54.5 s
Wall time: 54.5 s


In [3]:
page_dict = pd.Series(page_df.page_id.values,index=page_df.item_id).to_dict()
del(page_df)
page_dict[6199]

12

In [3]:
item_df.drop(columns='en_description', inplace=True)
item_df

,item_id,en_label
0,1,Universe
1,2,Earth
2,3,life
3,4,death
4,5,human
...,...,...
51450311,77257472,2dFGRS TGN256Z026
51450312,77257483,2dFGRS TGS171Z171
51450313,77257484,2dFGRS TGS373Z078
51450314,77257491,2dFGRS TGS374Z114


In [4]:
alias_df.rename(columns={'en_alias':'en_label'}, inplace=True)
alias_df

,item_id,en_label
0,1,Our Universe
1,1,The Universe
2,1,The Cosmos
3,1,cosmos
4,2,Blue Planet
...,...,...
6823019,77256516,"Wood Adams Building, Dunedin"
6823020,77256876,Flame-bearers of Welsh history being the outli...
6823021,77256970,Flame-bearers of Welsh history being the outli...
6823022,77257218,"6 Thorn Street, Caversham"


In [5]:
all_df = pd.concat([item_df, alias_df], ignore_index=True)
del(item_df)
del(alias_df)
all_df

,item_id,en_label
0,1,Universe
1,2,Earth
2,3,life
3,4,death
4,5,human
...,...,...
58273335,77256516,"Wood Adams Building, Dunedin"
58273336,77256876,Flame-bearers of Welsh history being the outli...
58273337,77256970,Flame-bearers of Welsh history being the outli...
58273338,77257218,"6 Thorn Street, Caversham"


In [6]:
all_df.dropna(inplace=True)
all_df

,item_id,en_label
0,1,Universe
1,2,Earth
2,3,life
3,4,death
4,5,human
...,...,...
58273335,77256516,"Wood Adams Building, Dunedin"
58273336,77256876,Flame-bearers of Welsh history being the outli...
58273337,77256970,Flame-bearers of Welsh history being the outli...
58273338,77257218,"6 Thorn Street, Caversham"


In [7]:
all_df.item_id = all_df.item_id.astype(int)
all_df.dtypes

item_id      int64
en_label    object
dtype: object

In [8]:
%%time
all_df.en_label = all_df.en_label.str.lower()
all_df.en_label = all_df.en_label.str.strip()
all_df

CPU times: user 33.6 s, sys: 2.46 s, total: 36.1 s
Wall time: 36.1 s


,item_id,en_label
0,1,universe
1,2,earth
2,3,life
3,4,death
4,5,human
...,...,...
58273335,77256516,"wood adams building, dunedin"
58273336,77256876,flame-bearers of welsh history being the outli...
58273337,77256970,flame-bearers of welsh history being the outli...
58273338,77257218,"6 thorn street, caversham"


In [9]:
%%time
#Very Slow, go grab some snacks, put on some netflix and come back in ~1 to 1.5 hours
all_group = all_df.groupby('en_label')
all_item = all_group['item_id'].apply(lambda group_series: pd.unique(group_series).tolist())
#del(all_df)
all_item

CPU times: user 1h 29min 1s, sys: 5min 1s, total: 1h 34min 2s
Wall time: 1h 27min 38s


en_label
!                                               [4540205, 66092288, 120976, 166764]
! -attention-                                                             [8290256]
! that bastard is trying to steal our gold !                             [60669584]
! that dick trying to steal our gold !                                   [60669584]
!!                                                                       [12366011]
                                                               ...                 
🧺                                                                          [201097]
🧿                                                                          [582742]
𠃍                                                                        [55900012]
𥫗                                                                        [55885207]
𪜈                                                                        [11273367]
Name: item_id, Length: 43618069, dtype: object

In [10]:
%%time
#takes ~3 min
with open("item_dict_full.p","wb") as f:
    pickle.dump(all_item, f, protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 49.1 s, sys: 3.84 s, total: 52.9 s
Wall time: 53 s


Run this portion if you want a filtered dictionary with page_ids.  
Not all item_ids will have page_ids, the following code will create a double dictionary with item and page ids for each en_label and drop all en_label's without a page_id.

In [7]:
all_df['page_id'] = [page_dict.get(i) for i in all_df.item_id]
all_df.dropna(inplace=True)
all_df.page_id = all_df.page_id.astype(int)
all_df

,item_id,en_label,page_id
0,1,Universe,31880.0
1,2,Earth,9228.0
2,3,life,18393.0
3,4,death,8221.0
4,5,human,682482.0
...,...,...,...
58273335,77256516,"Wood Adams Building, Dunedin",NaN
58273336,77256876,Flame-bearers of Welsh history being the outli...,NaN
58273337,77256970,Flame-bearers of Welsh history being the outli...,NaN
58273338,77257218,"6 Thorn Street, Caversham",NaN


In [ ]:
%%time
#Very Slow, go grab some snacks, put on some netflix and come back in ~30 min
all_group = all_df.groupby('en_label')
all_item = all_group['item_id'].apply(lambda group_series: pd.unique(group_series).tolist())
all_page = all_group['page_id'].apply(lambda group_series: pd.unique(group_series).tolist())
del(all_df)
all_group_df = pd.concat([all_item, all_page], axis=1)
all_group_df

In [ ]:
all_dict = all_group_df.to_dict()
all_dict

In [ ]:
%%time
#takes ~3 min
with open("item_dict_filtered.p","wb") as f:
    pickle.dump(all_dict, f, protocol=pickle.HIGHEST_PROTOCOL)